# VGG-Face Keras model

## Architecture

Details about the network architecture can be found in the following [paper](http://www.robots.ox.ac.uk/~vgg/software/vgg_face/):

```
Deep Face Recognition
O. M. Parkhi, A. Vedaldi, A. Zisserman
British Machine Vision Conference, 2015
```

Keras version of this network architecture can be found in https://gist.github.com/EncodeTS/6bbe8cb8bebad7a672f0d872561782d9



In [ ]:
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from PIL import Image
import numpy as np

def vgg_face(weights_path=None):
    img = Input(shape=(3, 224, 224), name='input')

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Convolution2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Convolution2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Convolution2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Convolution2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Convolution2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Convolution2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Convolution2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Convolution2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Convolution2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Convolution2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Convolution2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Convolution2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Convolution2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

    fc6 = Convolution2D(4096, 7, 7, activation='relu', name='fc6')(pool5)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Convolution2D(4096, 1, 1, activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    fc8 = Convolution2D(2622, 1, 1, name='fc8')(fc7_drop)
    flat = Flatten()(fc8)
    out = Activation('softmax')(flat)

    model = Model(input=img, output=out)

    if weights_path:
        model.load_weights(weights_path)

    return model

## Weights
The weights of the pretrained network can be downloaded [here](https://drive.google.com/open?id=0B4ChsjFJvew3NkF0dTc1OGxsOFU).

In [ ]:
# Load pretrained model
from keras import backend as K
K.set_image_data_format('channels_first')

model_fn = 'vgg_face_weights.h5'
model = vgg_face(model_fn)
model.summary()

## Test pretrained model

In [ ]:
# download a face image  
#!wget https://cloud.githubusercontent.com/assets/16507370/17052193/ef7c8098-502e-11e6-9fc4-ebcb085ca0a8.jpg -O test_img.jpg

im = Image.open('test_img.jpg')
im = im.resize((224,224))
im = np.array(im).astype(np.float32)
im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)
    
out = model.predict(im)
print(out[0][0])

## Save graphs

In [ ]:
# Save Keras version - h5
keras_model_fn = "keras_vgg_face_graph.h5"
model.save(keras_model_fn)

# Save tensorflow version - pb
tensorflow_model_fn = "tensorflow_vgg_face_graph.pb"
!python3.7 ../keras_to_tensorflow/keras_to_tensorflow.py --input_model=$keras_model_fn --output_model=$tensorflow_model_fn
# Note: keras_to_tensorflow.py is from the repo https://github.com/amir-abdi/keras_to_tensorflow
